In [6]:
import pandas as pd
import numpy  as np

np.random.seed(2018)

feature_names = ['Age','Pclass','Embarked','Sex']

# load data sets 
train = pd.read_csv('./titanic/train.csv', usecols =['Survived','PassengerId'] + feature_names)
test  = pd.read_csv('./titanic/test.csv',  usecols =['PassengerId'] + feature_names )

# combine train and test for joint processing 
test['Survived'] = np.nan
comb = pd.concat([ train, test ])
comb.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


,Age,Embarked,PassengerId,Pclass,Sex,Survived
0,22.0,S,1,3,male,0.0
1,38.0,C,2,1,female,1.0
2,26.0,S,3,3,female,1.0
3,35.0,S,4,1,female,1.0
4,35.0,S,5,3,male,0.0


In [9]:
print('Number of missing Embarked values ',comb['Embarked'].isnull().sum())
comb['Embarked'] = comb['Embarked'].fillna('S')
comb['Embarked'].unique()

Number of missing Embarked values  0


array(['S', 'C', 'Q'], dtype=object)

In [10]:
comb['NoAge'] = comb['Age'] == np.NAN
comb['Age'] =  comb['Age'].fillna(-1)
comb['Age'].hist(bins=100)
comb['Minor'] = (comb['Age']<14.0)&(comb['Age']>=0)

In [13]:
# one-hot encode Pclass
comb['P1'] = comb['Pclass'] == 1 
comb['P2'] = comb['Pclass'] == 2
comb['P3'] = comb['Pclass'] == 3

# one-hot encode Embarked
comb['ES'] = comb['Embarked'] == 'S' 
comb['EQ'] = comb['Embarked'] == 'Q'
comb['EC'] = comb['Embarked'] == 'C'

In [14]:
# Label encode Sex
comb['Sex'] = comb['Sex'].map({'male':0,'female':1})

# drop Pclass, Embarked and Age features
comb = comb.drop(columns=['Pclass','Embarked','Age'])
comb.head()

,PassengerId,Sex,Survived,NoAge,Minor,P1,P2,P3,ES,EQ,EC
0,1,0,0.0,False,False,False,False,True,True,False,False
1,2,1,1.0,False,False,True,False,False,False,False,True
2,3,1,1.0,False,False,False,False,True,True,False,False
3,4,1,1.0,False,False,True,False,False,True,False,False
4,5,0,0.0,False,False,False,False,True,True,False,False


In [15]:
df_train = comb.loc[comb['Survived'].isin([np.nan]) == False]
df_test  = comb.loc[comb['Survived'].isin([np.nan]) == True]

print(df_train.shape)
df_train.head()

(891, 11)


,PassengerId,Sex,Survived,NoAge,Minor,P1,P2,P3,ES,EQ,EC
0,1,0,0.0,False,False,False,False,True,True,False,False
1,2,1,1.0,False,False,True,False,False,False,False,True
2,3,1,1.0,False,False,False,False,True,True,False,False
3,4,1,1.0,False,False,True,False,False,True,False,False
4,5,0,0.0,False,False,False,False,True,True,False,False


In [16]:
print(df_test.shape)
df_test.head()

(418, 11)


,PassengerId,Sex,Survived,NoAge,Minor,P1,P2,P3,ES,EQ,EC
0,892,0,NaN,False,False,False,False,True,False,True,False
1,893,1,NaN,False,False,False,False,True,True,False,False
2,894,0,NaN,False,False,False,True,False,False,True,False
3,895,0,NaN,False,False,False,False,True,True,False,False
4,896,1,NaN,False,False,False,False,True,True,False,False


In [19]:
from sklearn.model_selection import GridSearchCV

In [24]:
feature_names = ['Sex','P1','P2','P3','EQ','ES','EC','NoAge','Minor']

from sklearn.svm import SVC
model = SVC()
param_grid = {'C':[1,2,5,10,20,50]} 
grs = GridSearchCV(model, param_grid=param_grid, cv = 10, n_jobs=1, return_train_score = False)
grs.fit(np.array(df_train[feature_names]), np.array(df_train['Survived']))

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=1,
             param_grid={'C': [1, 2, 5, 10, 20, 50]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [25]:
print("Best parameters " + str(grs.best_params_))
gpd = pd.DataFrame(grs.cv_results_)
print("Estimated accuracy of this model for unseen data:{0:1.4f}".format(gpd['mean_test_score'][grs.best_index_]))

Best parameters {'C': 1}
Estimated accuracy of this model for unseen data:0.8238


In [27]:
pred = grs.predict(np.array(df_test[feature_names]))

sub = pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':pred})
sub.to_csv('./titanic/AgeSexPclassEmbarked.csv', index = False, float_format='%1d')
sub.head()

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0
